In [1]:
!sudo pip install -q transformers --upgrade

In [2]:
from transformers import AutoModelForCausalLM

In [ ]:
model = AutoModelForCausalLM.from_pretrained("/data/mistral/query-to-mql/exp-10/nov-20/merged-model")
model.to_bettertransformer()

In [4]:
!sudo pip install -q git+https://github.com/huggingface/optimum.git

In [5]:
from optimum.pipelines import pipeline

In [6]:
onnx = pipeline("text-generation", model="/data/mistral/query-to-mql/exp-10/nov-20/merged-model", accelerator="ort")

ValueError: Task text-generation is not supported for the ONNX Runtime pipeline. Supported tasks are []

In [ ]:
question = "What's my name?"
context = "My name is Philipp and I live in Nuremberg."

pred = onnx_qa(question=question, context=context)